In [4]:
import os
import pandas as pd
import numpy as np
import glob

In [7]:
sub = 'pilot-52'
ses = 'ses-02'
battery = 'camcan'
task_source = 'emo-mem'
task = 'emotional-memory'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery)
csv_file = task_source + '_sub-52_' + ses + '.txt'
tsv_file = task + '_sub-52_' + ses + '.tsv'

In [14]:
file_df = pd.read_csv(os.path.join(datadir, csv_file), sep=' ')

In [15]:
file_df

Scanner pulse time: 163.4050
img_valence img_file           img_onset img_actual_onset img_onset_end obj_file               obj_onset obj_actual_onset obj_onset_end resp_key                   resp_onset
3           \Negative\9040.JPG 164.3621  164.3621         164.3622      \Objects\firetruck.bmp 166.3662  166.3662         166.3662      y                            168.0621
2           \Neutral\7100.JPG  174.8083  174.8083         174.8084      \Objects\curtain.bmp   176.8134  176.8134         176.8135      y                            180.6387
            \Neutral\7186.JPG  184.9533  184.9533         184.9534      \Objects\hairbrush.bmp 186.9580  186.9580         186.9580      y                            191.7407
1           \Positive\2311.JPG 195.0881  195.0881         195.0881      \Objects\chimney.bmp   197.0919  197.0919         197.0920      y                            202.7328
...                                                                                                                                                                       ...
2           \Neutral\2890.JPG  721.9112  721.9112         721.9113      \Objects\cap.bmp       723.9160  723.9160         723.9161      y                            725.9868
3           \Negative\2800.JPG 732.0501  732.0501         732.0501      \Objects\bag.bmp       734.0553  734.0553         734.0554      y                            737.0380
1           \Positive\2550.JPG 742.1827  742.1827         742.1827      \Objects\dinghy.bmp    744.1876  744.1876         744.1877      y                            748.7967
3           \Negative\9920.JPG 752.3085  752.3085         752.3085      \Objects\paperclip.bmp 754.3143  754.3143         754.3143      y                            756.6394
            \Negative\6230.JPG 762.4369  762.4369         762.4370      \Objects\cherries.bmp  764.4388  764.4388         764.4388      y                            764.4878

[61 rows x 1 columns]

In [16]:
# In the csv file extracted from the E-Prime txt file, 
# run onset is the 'GetReady.OffsetTime' column of last row, 
# and all other columns of this row are empty.
run_start = (file_df.loc[file_df['img_valence']=='Scanner'])['img_actual_onset']
run_start = run_start.values[0]

KeyError: 'img_valence'

In [26]:
run_start

2083.9232

In [5]:
# Find the trial-related rows
trials = file_df['img_valence'] != 'Scanner'
negative_trials = file_df['img_valence'] == '3'
neutral_trials = file_df['img_valence'] == '2'
positive_trials = file_df['img_valence'] == '1'

In [ ]:
print(len(trials), sum(trials))

In [ ]:
negative_trials

In [6]:
# Trial/Image onsets are in 'img_onset'
img_onsets = file_df.loc[trials]['img_onset'].astype(float)

# Object onsets are in 'obj_onset'
obj_onsets = file_df.loc[trials]['obj_onset'].astype(float)

# Concatenate the two to get the onsets array
trial_onsets = np.vstack((img_onsets, obj_onsets)).flatten('F')

## Create durations array
# In the task, the image is presented for 8 s. 2 s after image onset, 
# the object is presented, and stays on screen for the remaining 6 s.
img_durations = np.repeat(8,sum(trials))
obj_durations = np.repeat(6,sum(trials))
# Interlace the two durations to create the durations array
durations = (np.vstack((img_durations,obj_durations))).flatten('F')

In [29]:
trial_onsets

array([2084.6274, 2086.6325, 2094.7642, 2096.7694, 2104.9041, 2106.9089,
       2115.0401, 2117.0449, 2125.1764, 2127.1813, 2135.3149, 2137.3208,
       2145.447 , 2147.4525, 2155.5867, 2157.5885, 2165.7209, 2167.7255,
       2175.8589, 2177.8635, 2185.9986, 2188.0033, 2196.1352, 2198.1406,
       2206.2728, 2208.2775, 2216.4155, 2218.4202, 2226.5516, 2228.5566,
       2236.6907, 2238.6957, 2246.8201, 2248.8249, 2256.9486, 2258.9536,
       2267.0867, 2269.0916, 2277.2231, 2279.2278, 2287.3662, 2289.371 ,
       2297.5043, 2299.5074, 2307.6349, 2309.6397, 2317.7702, 2319.7748,
       2327.9054, 2329.9102, 2338.0414, 2340.0461, 2348.1788, 2350.1834,
       2358.315 , 2360.3196, 2368.4498, 2370.4527, 2378.5864, 2380.5913,
       2388.7299, 2390.7358, 2398.8662, 2400.8712, 2409.0038, 2411.0089,
       2419.1412, 2421.1462, 2429.2786, 2431.2831, 2439.4159, 2441.4209,
       2449.5454, 2451.5503, 2459.687 , 2461.6921, 2469.824 , 2471.8267,
       2479.9621, 2481.9671, 2490.1016, 2492.1064, 

In [7]:
## Finally, create the trial_types array. This will be an interlacing
## of the 'img_valence' vector for images and 'object' for objects
img_types = np.array(np.ones(len(trials)), dtype=np.object_)
img_types[negative_trials] = 'negative_image'
img_types[neutral_trials] = 'neutral_image'
img_types[positive_trials] = 'positive_image'
img_types = img_types[1:]

In [31]:
obj_types = np.repeat('object', sum(trials))

In [32]:
trial_types = np.vstack((img_types,obj_types)).flatten('F')

In [33]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start), 'duration':durations, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_file), sep='\t', float_format='%0.3f', index=False)